In [3]:
%matplotlib qt
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
import glob
import pandas as pd
import time

In [4]:
def MaskProcessing(image_path, output_path):
    coords = []
    action = {"next": False, "quit": False}

    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    img_scaled = (img / 16).astype("uint8")

    def apply_gamma_correction(image, gamma=2.2):
        img_float = image / 255.0
        img_gamma = np.power(img_float, 1.0 / gamma)
        return (img_gamma * 255).astype("uint8")

    img_display = apply_gamma_correction(img_scaled)
    img_rgb_display = cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB)

    def mask_region_and_save():
        mask = np.ones_like(img_scaled, dtype=np.uint8) * 255
        pts = np.array([coords], dtype=np.int32)
        cv2.fillPoly(mask, pts, (0, 0, 0))
        masked_img = cv2.bitwise_and(img_scaled, mask)
        cv2.imwrite(output_path, masked_img)
        print(f"Saved masked image to: {output_path}")
        plt.figure()
        plt.imshow(cv2.cvtColor(masked_img, cv2.COLOR_BGR2RGB))
        plt.title("Masked Image (region blacked out)")
        plt.axis("off")
        plt.show()

    def onclick(event):
        if event.xdata is not None and event.ydata is not None and len(coords) < 4:
            x, y = int(event.xdata), int(event.ydata)
            coords.append((x, y))
            ax.plot(x, y, 'ro')
            ax.text(x + 5, y - 5, f"{len(coords)}", color="red", fontsize=12)
            fig.canvas.draw()

            if len(coords) == 4:
                mask_region_and_save()
                ax.set_title("Press Enter to continue, q to quit")
                fig.canvas.draw()

    def onkey(event):
        if event.key == 'r':
            coords.clear()
            ax.clear()
            ax.imshow(img_rgb_display)
            ax.set_title("Click 4 points (Press 'r' to reset)")
            fig.canvas.draw()
            print("Reset points")
        elif event.key == 'enter':
            action["next"] = True
            plt.close()
        elif event.key == 'q':
            action["quit"] = True
            plt.close()

    fig, ax = plt.subplots()
    ax.imshow(img_rgb_display)
    ax.set_title("Click 4 points (Press 'r' to reset)")
    fig.canvas.mpl_connect("button_press_event", onclick)
    fig.canvas.mpl_connect("key_press_event", onkey)
    plt.show()

    # 🔽 明示的に「Enterが押される」まで待つ
    while not action["next"] and not action["quit"]:
        time.sleep(0.1)  # CPU負荷を避けるためスリープ

    return output_path, action


In [5]:
input_dir = "../maskpre"
output_dir = "../histpre"
os.makedirs(output_dir, exist_ok=True)

image_paths = glob.glob(os.path.join(input_dir, "*.png"))

for image_path in image_paths:
    filename = os.path.splitext(os.path.basename(image_path))[0]
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    if img is None:
        print(f"Failed to read {image_path}")
        continue

    # 黒以外の領域をマスク（mask: 0 or 255）
    mask = cv2.inRange(img, (1, 1, 1), (255, 255, 255))
    valid_mask = mask.flatten() > 0

    # ===== RGB ヒストグラム =====
    norm_img = img.astype('float32') / 255.0
    plt.figure(figsize=(10, 4))
    for i, color in enumerate(('r', 'g', 'b')):
        hist = cv2.calcHist([norm_img], [i], mask, [256], [0, 1])
        plt.plot(hist, color=color, label=f"{color.upper()} channel")
    plt.title(f"RGB Histogram: {filename}")
    plt.xlabel("Intensity")
    plt.ylabel("Pixel Count")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{filename}_rgb_hist.png"))
    plt.close()

    # ===== 2D ヒストグラム（rg空間） =====
    R = img[:, :, 0].astype('float32')
    G = img[:, :, 1].astype('float32')
    B = img[:, :, 2].astype('float32')
    total = R + G + B + 1e-6
    r = R / total
    g = G / total
    r_flat = r.flatten()[valid_mask]
    g_flat = g.flatten()[valid_mask]

    hist2d, _, _ = np.histogram2d(r_flat, g_flat, bins=[50, 50], range=[[0, 1], [0, 1]])
    presence_mask = hist2d > 0

    plt.figure(figsize=(6, 6))
    plt.imshow(presence_mask.T, origin='lower', cmap='gray',
               extent=[0, 1, 0, 1], aspect='auto')
    plt.colorbar(label='Pixel Exists (True/False)')
    plt.xlabel('r = R / (R+G+B)')
    plt.ylabel('g = G / (R+G+B)')
    plt.title(f"2D Hist (rg mask): {filename}")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{filename}_rg_hist.png"))
    plt.close()

    # presence_mask を CSV 保存
    pd.DataFrame(presence_mask.astype(int)).to_csv(
        os.path.join(output_dir, f"{filename}_rg_mask.csv"), index=False
    )

    print(f"Saved histograms for: {filename}")
    print(f"{filename}: max(r + g) = {np.max(r_flat + g_flat)}")


Saved histograms for: 8D5U5524_masked
8D5U5524_masked: max(r + g) = 0.9374998807907104
Saved histograms for: 8D5U5525_masked
8D5U5525_masked: max(r + g) = 0.9374998807907104
Saved histograms for: 8D5U5526_masked
8D5U5526_masked: max(r + g) = 0.8888887763023376
Saved histograms for: 8D5U5527_masked
8D5U5527_masked: max(r + g) = 0.9411764144897461
Saved histograms for: 8D5U5528_masked
8D5U5528_masked: max(r + g) = 0.9333332777023315


In [34]:
# CSVファイルを指定（パスを適宜変更してください）
file_path = '../histpre/8D5U5524_masked_rg_mask.csv'  # 例: 'image_data.csv'
data = np.loadtxt(file_path, delimiter=",")

# 画像として表示
plt.imshow(np.flipud(data.T), cmap='gray', vmin=0, vmax=1)

plt.axis('off')  # 枠や目盛りを非表示
plt.show()


In [47]:
input_dir = "../maskpre"
output_dir = "../teacher"

image_paths = glob.glob(os.path.join(input_dir, "*.png"))

for image_path in image_paths:
    filename = os.path.splitext(os.path.basename(image_path))[0]
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    if img is None:
        print(f"Failed to read {image_path}")
        continue

    # 黒以外の領域をマスク（mask: 0 or 255）
    mask = cv2.inRange(img, (1, 1, 1), (255, 255, 255))
    valid_mask = mask.flatten() > 0

    # ===== RGB ヒストグラム =====
    norm_img = img.astype('float32') / 255.0
    plt.figure(figsize=(10, 4))
    for i, color in enumerate(('r', 'g', 'b')):
        hist = cv2.calcHist([norm_img], [i], mask, [256], [0, 1])
        plt.plot(hist, color=color, label=f"{color.upper()} channel")
    plt.title(f"RGB Histogram: {filename}")
    plt.xlabel("Intensity")
    plt.ylabel("Pixel Count")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{filename}_rgb_hist.png"))
    plt.close()

    # ===== 2D ヒストグラム（rg空間） =====
    R = img[:, :, 0].astype('float32')
    G = img[:, :, 1].astype('float32')
    B = img[:, :, 2].astype('float32')
    total = R + G + B + 1e-6
    r = R / total
    g = G / total
    r_flat = r.flatten()[valid_mask]
    g_flat = g.flatten()[valid_mask]

    hist2d, _, _ = np.histogram2d(r_flat, g_flat, bins=[50, 50], range=[[0, 1], [0, 1]])
    presence_mask = hist2d > 0

    plt.figure(figsize=(6, 6))
    plt.imshow(presence_mask.T, origin='lower', cmap='gray',
               extent=[0, 1, 0, 1], aspect='auto')
    plt.colorbar(label='Pixel Exists (True/False)')
    plt.xlabel('r = R / (R+G+B)')
    plt.ylabel('g = G / (R+G+B)')
    plt.title(f"2D Hist (rg mask): {filename}")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{filename}_rg_hist.png"))
    plt.close()

    # presence_mask を CSV 保存
    pd.DataFrame(presence_mask.astype(int)).to_csv(
        os.path.join(output_dir, f"{filename}_rg_mask.csv"), index=False
    )

    print(f"Saved histograms for: {filename}")
    print(f"{filename}: max(r + g) = {np.max(r_flat + g_flat)}")


In [6]:
def pretreatment():
    input_dir = "../datapre"
    outputmask_dir = "../maskpre"
    outputhist_dir = "../histpre"
    outputtea_dir = "../teacher"
    
    image_paths = glob.glob(os.path.join(input_dir, "*.png"))

    for image_path in image_paths:
        filename = os.path.splitext(os.path.basename(image_path))[0]
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

        if img is None:
            print(f"Failed to read {image_path}")
            continue
        
        # 拡張子を分離
        name, ext = os.path.splitext(image_path)
        
        # "_masked" を挿入
        image_masked_path = f"{name}_masked{ext}"
        
        print(MaskProcessing(image_path,outputmask_dir + image_masked_path))
        # CreateHistgram()
        # TeacherProcessing()
        # CreateHistgram()
        # RemoveEnd()


In [ ]:
import os
import glob
import cv2

def pretreatment():
    input_dir = "../datapre"
    outputmask_dir = "../maskpre"
    outputhist_dir = "../histpre"
    outputtea_dir = "../teacher"

    image_paths = glob.glob(os.path.join(input_dir, "*.png"))

    for image_path in image_paths:
        filename = os.path.splitext(os.path.basename(image_path))[0]
        image_masked_path = os.path.join(outputmask_dir, f"{filename}_masked.png")

        result_path, action = MaskProcessing(image_path, image_masked_path)
        print(f"保存結果: {result_path}")
        if action["quit"]:
            print("ユーザーが 'q' を押したため処理を中断します。")
            break
        elif not action["next"]:
            print("Enter を押すまで進みません（通常はここには来ない）")

        # TODO: 他の処理（ヒストグラム、教師データなど）
        # CreateHistgram()
        # TeacherProcessing()
        # RemoveEnd()

# 実行
pretreatment()
